In [1]:
import warnings
warnings.simplefilter('ignore')

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Generative-Adversial Network

**generador**

In [2]:
import keras
from keras import layers
import numpy as np


latent_dim = 32

height = 32
width = 32
channels = 3

Using TensorFlow backend.


In [3]:
generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', 
                  padding='same')(x)


generator = keras.models.Model(generator_input, x)


generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

**discriminante**

In [4]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)


discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [5]:
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)


discriminator.compile(optimizer=discriminator_optimizer,
                     loss='binary_crossentropy')

**adversial network**

In [6]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,)) 

gan_output = discriminator(generator(gan_input)) 

gan = keras.models.Model(gan_input, gan_output)


gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, 
                                         clipvalue=1.0, 
                                         decay=1e-8) 

gan.compile(optimizer=gan_optimizer, 
            loss='binary_crossentropy')

**entrenando la red**

In [7]:
import os
from keras.preprocessing import image

In [8]:
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

In [9]:
x_train = x_train[y_train.flatten() == 6]  # ranas

x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.


iterations = 1000
batch_size = 20
save_dir = 'data/gan'

In [10]:
start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
                                        latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size 
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)),
                         np.zeros((batch_size, 1))])
    
    labels += 0.05 * np.random.random(labels.shape)
    d_loss = discriminator.train_on_batch(combined_images, labels)
    random_latent_vectors = np.random.normal(size=(batch_size,
                                                   latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors,
                                misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        
    if step % 100 == 0:
        gan.save_weights('data/gan/gan.h5')
        
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
        print ()
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,
                      'generated_frog' + str(step) + '.png'))
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,
                      'real_frog' + str(step) + '.png'))

discriminator loss: 0.70559543
adversarial loss: 0.6765305
discriminator loss: 0.50950974
adversarial loss: 2.0949702
discriminator loss: 0.7140133
adversarial loss: 0.74036235
discriminator loss: 0.7049266
adversarial loss: 0.73278207
discriminator loss: 0.6961688
adversarial loss: 0.98554593
discriminator loss: 0.7179936
adversarial loss: 0.76294136
discriminator loss: 0.68202376
adversarial loss: 0.68643486
discriminator loss: 0.6905818
adversarial loss: 0.7388077


KeyboardInterrupt: 